<a href="https://colab.research.google.com/github/anushiya-thevapalan/test/blob/master/IMDB_Sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download the needed Files

In order to work on this notebook we need to download 2 files. 

1.   Glove word embedding
2.   Dataset


## Glove word embedding
GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space.

http://nlp.stanford.edu/data/glove.6B.zip


## Large Movie Review Dataset
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well. Raw text and already processed bag of words formats are provided. See the README file contained in the release for more details. 

http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz




**Download Glove word embedding**

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-03-01 13:31:15--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-03-01 13:31:15--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  37.6MB/s    in 32s     

2019-03-01 13:31:47 (25.7 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



**Unzip the glove.6B.zip**

In [ ]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


Download  Movie Review Dataset

In [ ]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2019-03-01 13:33:25--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  12.9MB/s    in 5.3s    

2019-03-01 13:33:31 (15.1 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



**Unzip the Files**

In [ ]:
!tar xf aclImdb_v1.tar.gz

**Importing required libraries**

In [ ]:
import os
import re
import string
import numpy as np
from keras import Model
from keras.utils import to_categorical, plot_model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, concatenate, Bidirectional
from keras.layers.embeddings import Embedding
import matplotlib.pyplot as plt

Using TensorFlow backend.


**Setting the max length**

In [ ]:
maxLen = 200

**Load the data**

In [ ]:
def loadData():
    # load File names
    trainPosFiles = os.listdir("./aclImdb/train/pos")
    trainNegFiles = os.listdir("./aclImdb/train/neg")
    testPosFiles = os.listdir("./aclImdb/test/pos")
    testNegFiles = os.listdir("./aclImdb/test/neg")

    #load positive, negative files from the directory

    trainPos = []
    trainNeg = []
    testPos = []
    testNeg = []

    for i in range(len(trainPosFiles)):
        with open("./aclImdb/train/pos/" + trainPosFiles[i], "r") as myfile:
            # Lower the text, preprocess the text
            line = preprocess((myfile.readlines()[0]).lower())
            trainPos.append(line)

    for i in range(len(trainNegFiles)):
        with open("./aclImdb/train/neg/" + trainNegFiles[i], "r") as myfile:
            # Lower the text, preprocess the text
            line = preprocess((myfile.readlines()[0]).lower())
            trainNeg.append(line)

    for i in range(len(trainPosFiles)):
        with open("./aclImdb/test/pos/" + testPosFiles[i], "r") as myfile:
            # Lower the text, preprocess the text
            line = preprocess((myfile.readlines()[0]).lower())
            testPos.append(line)

    for i in range(len(trainNegFiles)):
        with open("./aclImdb/test/neg/" + testNegFiles[i], "r") as myfile:
            # Lower the text, preprocess the text
            line = preprocess((myfile.readlines()[0]).lower())
            testNeg.append(line)

    #merge Positive and Negative Datasets
    trainX = trainPos + trainNeg
    testX = testPos + testNeg

    #preparing the labels for the dataset and  onehot encode
    trainY = to_categorical([1 if i < len(trainPos) else 0 for i in range(len(trainX))],num_classes=2)
    testY = to_categorical([1 if i < len(testPos) else 0 for i in range(len(testX))],num_classes=2)

    return np.array(trainX), np.array(trainY), np.array(testX), np.array(testY)

 **Preprocess text**

In [ ]:
#Function to preprocess the text
def preprocess(txt):
    def removeHTML(txt):
        #remove the Html in the text
        pattern = r'(<(?P<tag>[a-zA-Z0-9]+)>.*?</(?P=tag)>)'
        return re.sub(pattern, ' ', txt, flags=re.MULTILINE)

    def removeXml(txt):
        # remove the XML in the text
        pattern = r'/[a-zA-Z-_/]*\.xml'
        return re.sub(pattern, ' ', txt, flags=re.MULTILINE)

    def removeContinousFullstops(txt):
        # remove the Continous fullstop Eg: "......" with one fullstop.
        return re.sub('\.\.+', ' . ', txt,flags=re.MULTILINE)

    def removeNumbers(txt):
        # Remove the digits .
        pattern = r'\d+'
        return re.sub(pattern, ' . ', txt, flags=re.MULTILINE)

    def removePunctuationwithoutdot(txt):
        remove = string.punctuation
        remove = remove.replace(".", "")  # don't remove hyphens
        pattern = r"[{}]".format(remove)  # create the pattern
        return  re.sub(pattern, "", txt)

    txt=removeHTML(txt)
    txt =removeXml(txt)
    txt =removeContinousFullstops(txt)
    txt =removeNumbers(txt)
    txt =removePunctuationwithoutdot(txt)
    return txt

**Load the training and testing samples**

In [ ]:
trainX, trainY, testX, testY = loadData()

In [ ]:
print("Number of training samples : ", len(trainY))
print("Number of testing samples : ", len(trainY))

Number of training samples :  25000
Number of testing samples :  25000


**Load the Glove Vectors**

In [ ]:
# Function to read the Glove word embedding
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)

        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map


print("Loading Glove Vectors...")
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.200d.txt')

Loading Glove Vectors...


**Function to convert sentences_to_indices**

In [ ]:
#function to convert sentences_to_indices

def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4).

    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this.

    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """

    m = X.shape[0]  # number of training examples

    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m, max_len))

    for i in range(m):  # loop over training examples

        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = [w.lower() for w in X[i].split()]

        # Initialize j to 0
        j = 0

        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.

            try:
                X_indices[i, j] = word_to_index[w]
            except Exception as e:
                X_indices[i, j] = 0

            # Increment j to j + 1
            j += 1
            if j == 199:
                break

    return X_indices

**Make Embedding Layer**

In [ ]:
# pretrained_embedding_layer

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.

    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """

    vocab_len = len(word_to_index) + 1  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]  # define dimensionality of your GloVe word vectors (= 50)

    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))

    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False.
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))

    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])

    return embedding_layer


**Build The Model**

In [ ]:
#buildModel

def buildModel(input_shape, word_to_vec_map, word_to_index):
    """
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """

    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(input_shape, dtype='int32')

    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)

    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)

    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = Bidirectional(LSTM(128, return_sequences=True))(embeddings)

    # Add dropout with a probability of 0.5
    X = Dropout(0.2)(X)
    X = Bidirectional(LSTM(128, return_sequences=False))(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.2)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(2)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)

    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)

    return model

**MOdel Summary**

In [ ]:
print("Building The Model...")
model = buildModel((maxLen,), word_to_vec_map, word_to_index)
model.summary()
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)



Building The Model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 200, 200)          80000200  
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200, 256)          336896    
_________________________________________________________________
dropout_3 (Dropout)          (None, 200, 256)          0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 256)               394240    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                

**Train the Model**

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
X_train_indices = sentences_to_indices(trainX, word_to_index, maxLen)
history=model.fit(X_train_indices, trainY,validation_split=0.1, epochs=40 ,batch_size=50 , shuffle=True)

Instructions for updating:
Use tf.cast instead.
Train on 22500 samples, validate on 2500 samples
Epoch 1/40
22500/22500 [==============================] - 1142s 51ms/step - loss: 0.5746 - acc: 0.6936 - val_loss: 0.5505 - val_acc: 0.7796
Epoch 2/40
22500/22500 [==============================] - 1140s 51ms/step - loss: 0.4341 - acc: 0.8023 - val_loss: 0.3675 - val_acc: 0.8424
Epoch 3/40
22500/22500 [==============================] - 1134s 50ms/step - loss: 0.3656 - acc: 0.8409 - val_loss: 0.2728 - val_acc: 0.8744
Epoch 4/40
22500/22500 [==============================] - 1138s 51ms/step - loss: 0.3313 - acc: 0.8580 - val_loss: 0.3429 - val_acc: 0.8728
Epoch 5/40
22500/22500 [==============================] - 1132s 50ms/step - loss: 0.3076 - acc: 0.8688 - val_loss: 0.4985 - val_acc: 0.7724
Epoch 6/40
22500/22500 [==============================] - 1134s 50ms/step - loss: 0.2808 - acc: 0.8826 - val_loss: 0.4470 - val_acc: 0.8072
Epoch 7/40
22500/22500 [==============================] - 1135s

**Visualize the Accuracy and the loss**

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]

    if len(loss_list) == 0:
        print('Loss is missing in history')
        return

        ## As loss always exists
    epochs = range(1, len(history.history[loss_list[0]]) + 1)

    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b',
                 label='Training loss (' + str(str(format(history.history[l][-1], '.5f')) + ')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g',
                 label='Validation loss (' + str(str(format(history.history[l][-1], '.5f')) + ')'))

    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b',
                 label='Training accuracy (' + str(format(history.history[l][-1], '.5f')) + ')')
    for l in val_acc_list:
        plt.plot(epochs, history.history[l], 'g',
                 label='Validation accuracy (' + str(format(history.history[l][-1], '.5f')) + ')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig('accuracy.png', bbox_inches='tight')
    
plot_history(history)